In [ ]:
import os
os.chdir(os.path.dirname("../"))

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%load_ext autoreload
%autoreload 2

# deepOF data exploration

Given a dataset, this notebook allows the user to 

* Load and process the dataset using deepof.data
* Visualize data quality with interactive plots
* Visualize training instances as multi-timepoint scatter plots with interactive configurations
* Visualize training instances as video clips with interactive configurations

In [ ]:
import deepof.data
import deepof.utils
import numpy as np
import pandas as pd
import ruptures as rpt
import tensorflow as tf

from ipywidgets import interact, widgets
from IPython import display
from matplotlib.animation import FuncAnimation
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
import seaborn as sns

### 1. Define and run project

In [ ]:
exclude_bodyparts = tuple([""])

In [ ]:
# Use deepof to load a project
proj = deepof.data.Project(
    path="./tests/test_examples/test_single_topview/",
    arena_dims=380,
    arena_detection="rule-based",
    exclude_bodyparts=exclude_bodyparts,
    enable_iterative_imputation=True,
    interpolate_outliers=True,
    smooth_alpha=5,
    exp_conditions={"test":"test_cond", "test2":"test_cond"},
).run()

### 2. Inspect dataset quality

In [ ]:
all_quality = pd.concat([tab for tab in proj.get_quality().values()])

In [ ]:
all_quality.boxplot(rot=45)
plt.ylim(0.99985, 1.00001)
plt.show()

In [ ]:
@interact(quality_top=(0.0, 1.0, 0.01))
def low_quality_tags(quality_top):
    pd.DataFrame(
        pd.melt(all_quality)
        .groupby("bodyparts")
        .value.apply(lambda y: sum(y < quality_top) / len(y) * 100)
    ).sort_values(by="value", ascending=False).plot.bar(rot=45)

    plt.xlabel("body part")
    plt.ylabel("Tags with quality under {} (%)".format(quality_top * 100))
    plt.tight_layout()
    plt.legend([])
    plt.show()

In the cell above, you see the percentage of labels per body part which have a quality lower than the selected value (0.50 by default) **before** preprocessing. The values are taken directly from DeepLabCut.

### 3. Get coordinates, distances and angles

And get speed, acceleration and jerk for each

In [ ]:
# Get coordinates, speeds, accelerations and jerks for positions
position_coords = proj.get_coords(center="Center", align="Spine_1")
position_speeds = proj.get_coords(center="Center", speed=1)
position_accels = proj.get_coords(center="Center", speed=2)
position_jerks = proj.get_coords(center="Center", speed=3)

In [ ]:
# Get coordinates, speeds, accelerations and jerks for distances
distance_coords = proj.get_distances()
distance_speeds = proj.get_distances(speed=1)
distance_accels = proj.get_distances(speed=2)
distance_jerks = proj.get_distances(speed=3)

In [ ]:
# Get coordinates, speeds, accelerations and jerks for angles
angle_coords = proj.get_angles()
angle_speeds = proj.get_angles(speed=1)
angle_accels = proj.get_angles(speed=2)
angle_jerks = proj.get_angles(speed=3)

### 4. Analyse smoothing

In [ ]:
@interact(test=position_coords.keys(), 
          bpart = set([i[0] for i in list(position_coords.values())[0].columns]), 
          max_obs=(100, 1000), 
          smooth_degree=widgets.IntSlider(min=1, max=10, value=13), 
          smooth_wlen=widgets.IntSlider(min=11, max=101, step=2, value=15))
def plot_smoothing(test, bpart, max_obs, smooth_degree, smooth_wlen):
    
    fig = plt.figure(figsize=(12, 8))
    
    no_smooth = deepof.data.Project(
    path="./tests/test_examples/test_single_topview/",
    arena_dims=380,
    arena_detection="rule-based",
    exclude_bodyparts=exclude_bodyparts,
    interpolate_outliers=True,
    smooth_alpha=0,
    ).run(verbose=0).get_coords(center="Center", align="Spine_1", align_inplace=True)
    
    no_smooth_coords = no_smooth[test].iloc[:max_obs, :].loc[:, bpart]
    smooth_coords = savgol_filter(x=no_smooth_coords, 
                                  window_length=smooth_wlen, 
                                  polyorder=smooth_degree, 
                                  axis=0)
    
    plt.plot(no_smooth_coords, linestyle="--", label="no smoothing")
    plt.plot(smooth_coords, color="red", label="savgol, poly={}, wlen={}".format(smooth_degree, smooth_wlen))
    
    plt.title("SavGol smoothing exploration")
    plt.xlabel("Time")
    plt.ylabel("Coordinate value")

    plt.legend()
    plt.tight_layout()
    plt.show()

### 5. Display training instances

In [ ]:
random_exp = np.random.choice(list(position_coords.keys()), 1)[0]


@interact(time_slider=(0.0, 100, 25), length_slider=(10, 100, 5))
def plot_mice_across_time(time_slider, length_slider):

    plt.figure(figsize=(10, 10))

    for bpart in position_coords[random_exp].columns.levels[0]:
        if bpart != "Center":
            sns.scatterplot(
                data=position_coords[random_exp].loc[
                    time_slider : time_slider + length_slider - 1, bpart
                ],
                x="x",
                y="y",
                label=bpart,
                palette=sns.color_palette("tab10"),
            )

    plt.title("Positions across time for centered data")
    plt.legend(
        fontsize=15,
        bbox_to_anchor=(1.5, 1),
        title="Body part",
        title_fontsize=18,
        shadow=False,
        facecolor="white",
    )

    plt.ylim(-100, 60)
    plt.xlim(-60, 60)
    plt.show()

The figure above is a multi time-point scatter plot. The time_slider allows you to scroll across the video, and the length_slider selects the number of time-points to include. The idea is to intuitively visualize the data that goes into a training instance for a given preprocessing setting.

In [ ]:
# Auxiliary animation functions


def plot_mouse_graph(instant_x, instant_y, ax, edges):
    """Generates a graph plot of the mouse"""
    plots = []
    for edge in edges:
        (temp_plot,) = ax.plot(
            [float(instant_x[edge[0]]), float(instant_x[edge[1]])],
            [float(instant_y[edge[0]]), float(instant_y[edge[1]])],
            color="#006699",
            linewidth=2.0,
        )
        plots.append(temp_plot)
    return plots


def update_mouse_graph(x, y, plots, edges):
    """Updates the graph plot to enable animation"""

    for plot, edge in zip(plots, edges):
        plot.set_data(
            [float(x[edge[0]]), float(x[edge[1]])],
            [float(y[edge[0]]), float(y[edge[1]])],
        )

In [ ]:
random_exp = np.random.choice(list(position_coords.keys()), 1)[0]
print(random_exp)


@interact(time_slider=(0.0, 90.0, 25), length_slider=(10, 100, 5))
def animate_mice_across_time(time_slider, length_slider):

    fig, ax = plt.subplots(1, 1, figsize=(10, 10))

    edges = deepof.utils.connect_mouse_topview()

    for bpart in exclude_bodyparts:
        if bpart:
            edges.remove_node(bpart)

    for limb in ["Left_fhip", "Right_fhip", "Left_bhip", "Right_bhip"]:
        edges.remove_edge("Center", limb)
        if ("Tail_base", limb) in list(edges.edges()):
            edges.remove_edge("Tail_base", limb)

    edges = edges.edges()

    data = position_coords[random_exp].loc[
        time_slider : time_slider + length_slider - 1, :
    ]

    data["Center", "x"] = 0
    data["Center", "y"] = 0

    init_x = data.xs("x", level=1, axis=1, drop_level=False).iloc[0, :]
    init_y = data.xs("y", level=1, axis=1, drop_level=False).iloc[0, :]

    plots = plot_mouse_graph(init_x, init_y, ax, edges)
    scatter = ax.scatter(x=np.array(init_x), y=np.array(init_y), color="#006699",)

    # Update data in main plot
    def animation_frame(i):
        # Update scatter plot
        x = data.xs("x", level=1, axis=1, drop_level=False).iloc[i, :]
        y = data.xs("y", level=1, axis=1, drop_level=False).iloc[i, :]

        scatter.set_offsets(np.c_[np.array(x), np.array(y)])
        update_mouse_graph(x, y, plots, edges)

        return scatter

    animation = FuncAnimation(
        fig, func=animation_frame, frames=length_slider, interval=75,
    )

    ax.set_title("Positions across time for centered data")
    ax.set_ylim(-90, 60)
    ax.set_xlim(-60, 60)
    ax.set_xlabel("x")
    ax.set_ylabel("y")

    video = animation.to_html5_video()
    html = display.HTML(video)
    display.display(html)
    plt.close()

The figure above displays exactly the same data as the multi time-point scatter plot, but in the form of a video (one training instance at the time).

### 6. Detect changepoints

In [ ]:
test_ruptures = np.array(position_coords["test"])

# Filter out dimensions with low variance, removing aligned axis
from sklearn.feature_selection import VarianceThreshold
test_ruptures = VarianceThreshold(threshold=1e-3).fit_transform(test_ruptures)
test_ruptures.shape

In [ ]:
%%time
rupture_model = rpt.BottomUp(model="normal", min_size=15, jump=1).fit(test_ruptures)

In [ ]:
%%time
result = rupture_model.predict(pen=1.0)
print("number of ruptures:", len(result))
rupt_lengths = np.array(result)[1:] - np.array(result)[:-1]
print("rupture length mean =", rupt_lengths.mean())
print("rupture length std =", rupt_lengths.std())
print("rupture length max =", rupt_lengths.max())
print("rupture length min =", rupt_lengths.min())

In [ ]:
cut = 500
cut = result[np.argmin(np.abs(np.array(result) - cut))]
print("cut:", cut)
rpt.display(test_ruptures[:cut], [i for i in result if i <= cut])
plt.show()

## 7. Dynamic Time Warping to a fixed length

In [ ]:
tt = proj.supervised_annotation()

In [ ]:
proj.get_coords(propagate_labels=True, 
                propagate_annotations=tt).preprocess(automatic_changepoints=False,
                                                     scale="standard", 
                                                     test_videos=0)[0].shape